In [24]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

In [25]:
load_dotenv()

True

In [26]:
api_key = os.getenv('OPENAI_API_KEY')

if not api_key:
    print("api key is not loaded")
else:
    print("api key is loaded")

api key is loaded


In [27]:
openai = OpenAI()

In [28]:
class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)

        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else 'No title found'
        if soup.body:
            for irrelevant in soup.body.find_all(['script', 'style', 'img', 'input']):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator='\n', strip=True)
        else:
            self.text = ''


In [29]:
system_prompt = """You are assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
response in markdown
"""

In [30]:
def user_prompt_for(website):
    user_prompt = f"you are looking at a website title {website.title}\n"
    user_prompt += f"""
    The contents of this website is as follows\n;
    please provide a short summary of this website in markdown.
    If it includes any news or announcements, then summarize these too.
    """
    user_prompt += website.text
    return user_prompt

In [31]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [32]:
def summarize(url):
    website = Website(url)
    messages = messages_for(website)
    
    res = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return res.choices[0].message.content

In [33]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [34]:
display_summary('https://www.canada.ca/en.html')

# Summary of Canada.ca

**Website Overview**  
Canada.ca is the official website of the Government of Canada, providing access to a wide range of services and information related to various sectors including jobs, immigration, travel, health, benefits, taxes, and the environment. The site serves as a resource for citizens and residents to find relevant government services, news, and announcements.

**Key Features and Services**  
- **Jobs:** Information on job finding, training, hiring, and permits.
- **Immigration and Citizenship:** Guidance for visiting, working, or studying in Canada, as well as applying for residency.
- **Travel and Tourism:** Resources for travel within Canada and abroad including advisories and passport information.
- **Business Support:** Information on starting a business, permits, and support services.
- **Benefits:** Details on employment insurance, pensions, and various forms of aid.
- **Health:** Coverage on healthcare topics, nutrition, and product safety.
- **Environment:** Information on climate, wildlife, and conservation efforts.

**Recent News and Announcements**  
- **Strengthening Border Security:** Overview of plans to enhance efficiency and security at borders.
- **Every Dollar Counts:** Program to help citizens determine eligibility for benefits and tax assistance.
- **Canadian Dental Care Plan:** Expansion of eligibility for dental care applications.
- **Black History Month:** Recognition of contributions by Black Canadians throughout February.
- **Energy Efficiency Initiatives:** Tips for reducing energy bills and improving home energy savings.
- **National Flag of Canada:** Celebratory acknowledgment of the flag’s 60th anniversary.
- **Wildfire Monitoring:** Introduction of the WildFireSat mission for daily wildfire monitoring.
- **International Responses:** Information on Canada's response to the crises in Ukraine and Israel, West Bank, and Gaza.
- **Invictus Games 2025:** Call to support Team Canada during the upcoming games.

This website acts as a central hub for government resources and current events relevant to Canadian citizens and residents.